In [1]:
import chromadb
client = chromadb.PersistentClient()

In [2]:
answers = client.create_collection(
    name="answers"
)

In [4]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel("../resource/comfort.xlsx")
df.sample(5)

,user,answer,class
12547,피해 다니면 좋아했던 마음이 좀 사라질까?,좋아하는 마음을 억지로 없애려고 하면 더 힘들어질 거예요. 오히려 그 사람에게 솔직...,연애
1257,나중에 뭐할까 고민이야,지금 하는 일 열심히 하다보면 길이 보일거예요~,일상
5470,신용카드 다 없애야겠어,신용카드는 혜택이 많아서 잘 쓰면 좋지만 씀씀이가 커지니 조심해야해요,일상
10422,변하지 않는 사람도 있겠지?,"네 있습니다. 물론 그 반대의 경우도 있죠. 변하지 않는 것과 변하는 것, 어느 쪽...",연애
9744,남자친구가 술주정 부려,포기하거나 이해하거나 둘 중 하나예요.,연애


In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
ids = []
metadatas = []
embeddings = model.encode(df['user'],
                          normalize_embeddings=True,
                          show_progress_bar=True)

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].user
    answer = row[1].answer
    
    metadata = {
        "query": query,
        "answer": answer
    }
    ids.append(str(index))
    metadatas.append(metadata)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

Batches: 100%|████████████████████████████████| 397/397 [01:23<00:00,  4.76it/s]
12701it [00:00, 33037.14it/s]
100%|███████████████████████████████████████████| 13/13 [00:08<00:00,  1.62it/s]


In [32]:
result = answers.query(
    query_embeddings=model.encode("자꾸만 눈물이 나와", normalize_embeddings=True).tolist(),
    n_results=1
)

result_df = pd.DataFrame(result['metadatas'][0])
result_df['distances'] = result['distances'][0]
result_df

,answer,query,distances
0,왜 울어요? 무슨 일 있어요?,갑자기 눈물 나,0.332244


In [38]:
result['metadatas'][0][0]['answer']

'왜 울어요? 무슨 일 있어요?'